## 모델 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00


In [3]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.lib.tokenization_kobert import KoBertTokenizer
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.lib import tokenization_kobert
import json
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [4]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [5]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai
with open('/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/save_src_tk.pickle','rb') as f:
    src_tokenizer = pickle.load(f)

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai


In [6]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

## 테스트데이터 불러오기

In [9]:
def read_input(path):
    with open(path, 'rb') as f:
        test_dict = json.load(f)
    test_standard = []
    test_dialect = []
    for pairs in test_dict['utterance']:
        test_standard.append(pairs['standard_form'])
        test_dialect.append(pairs['dialect_form'])


    
    return test_standard ,test_dialect

In [10]:
test_s ,test_d= read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/test_data_big.json")

In [11]:
def translate(text):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    tmp = trg_tokenizer.decode(output.cpu())
    return tmp

In [12]:
print("size of full : ",len(test_s))

size of full :  12451


In [16]:
for i in range(len(test_s)):
    sr_text = test_d[i] # dialect
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result :      ",tg_text)
    print("ground_truth :",test_s[i])
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


source_text :  언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
result :       언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터     
ground_truth : 언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터


iteration num :  0
source_text :  어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예
result :       어 명절 설 명절 추석 명절 이렇게 나누어서 해볼 거예요   
ground_truth : 어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요


source_text :  자 그믄 이제부터 얘기해 보게예
result :       자 그러면 이제부터 얘기해 봐요         
ground_truth : 자 그러면 이제부터 얘기해 봐요


source_text :  어 그믄 언니네 설 명절 때 음식 어떵 해
result :       어 그러면 언니네 설 명절 때 음식 어떻게 해       
ground_truth : 어 그러면 언니네 설 명절 때 음식 어떻게 해


source_text :  음식을 막 준비했던 기억이 나게
result :       음식을 막 준비했던 기억이 나겠어요        
ground_truth : 음식을 막 준비했던 기억이 나


source_text :  밀가룬지 쌀가룬지를 모르크라  근데 그거를 푹
result :       밀가루인지 쌀가루인지 모르겠어 근데 그거를 푹      
ground_truth : 밀가룬지 쌀가룬지를 모르겠어  근데 그거를 푹


source_text :  반죽을 해그내 날 납작하게
result :       반죽을 해서 날 납작하게             
ground_truth : 반죽을 해서 날 납작하게


source_text :  게 밀어 밀대로 밀어그내
result :       게 밀어 밀대로 밀어서            
ground_truth : 게 밀어 밀대로 밀어서


source

KeyboardInterrupt: ignored